In [1]:
import sys, os
os.chdir(os.path.dirname(os.path.abspath("./")))
print(os.getcwd())

# SCRIPT_DIR = os.path.abspath(".")
# print(SCRIPT_DIR)
# sys.path.append(SCRIPT_DIR)
# sys.path


/home/konstfed/Documents/Study/PMLDL/PMLDL-TextDetoxification


In [2]:

import yaml
from addict import Dict

from src.models import build_model
from src.inference import ToxicClassificationPipeline, BertPipeline
from src import preprocessing

sys.modules['preprocessing'] = preprocessing

/home/konstfed/Documents/Study/PMLDL/PMLDL-TextDetoxification/toxic-venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
with open("configs/sentence_toxic_cls/hard_labels.yaml", "r") as f:
    hard_config = Dict(yaml.safe_load(f))

with open("configs/sentence_toxic_cls/soft_labels.yaml", "r") as f:
    soft_config = Dict(yaml.safe_load(f))

with open("configs/sentence_toxic_cls/distilbert_inference.yaml", "r") as f:
    bert_config = Dict(yaml.safe_load(f))


soft_cls = ToxicClassificationPipeline(soft_config) # BCEloss 0.45 on test
hard_cls = ToxicClassificationPipeline(hard_config) # BCEloss 0.46 on test
bert_cls = BertPipeline(bert_config) # BCEloss 0.05 on test

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.weight', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
texts2analyze = [
    "Dear professor, For starters, We would like to emphasize that we greatly appreciate your collaboration and understanding so far: Extending the deadline of the first assignment, agreeing to the online format, answering our questions in the group chat (just to name a few). Furthermore, we cannot stress enough that this is by no means personal. The general attendance rate is low, due to multiple incremental factors like students having to work to support themselves, regardless of the course in question and if attendance is mandatory or not. We would also like to note that last week was midterms week with overlapping deadlines and that we don’t have a rich mathematical background like the students of MIPT, due to the industrial/practical nature of most courses at Innopolis; so, sometimes it is wiser to skip lecture and dedicate more time to go over not only lecture materials but necessary prerequisites to fill our knowledge gaps. It’s also important to note that if this course wasn’t important to us, there wouldn’t be that many students signing up for it (this is double the usual amount). We’re just doing the best we can with the starting state we were in and with the other workload of university/life.",
    "Fucking kill you, For starters, We would like to emphasize that we greatly appreciate your collaboration and understanding so far: Extending the deadline of the first assignment, agreeing to the online format, answering our questions in the group chat (just to name a few). Furthermore, we cannot stress enough that this is by no means personal. The general attendance rate is low, due to multiple incremental factors like students having to work to support themselves, regardless of the course in question and if attendance is mandatory or not. We would also like to note that last week was midterms week with overlapping deadlines and that we don’t have a rich mathematical background like the students of MIPT, due to the industrial/practical nature of most courses at Innopolis; so, sometimes it is wiser to skip lecture and dedicate more time to go over not only lecture materials but necessary prerequisites to fill our knowledge gaps. It’s also important to note that if this course wasn’t important to us, there wouldn’t be that many students signing up for it (this is double the usual amount). We’re just doing the best we can with the starting state we were in and with the other",
    "If you have not time to grade our works, why you think that we have enough time to write 7 pages of report with full-time work?",
    "Also no bonus points for late grading. Half of works are not graded. The worst course at this moment",
]

In [10]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [11]:
for i, text in enumerate(texts2analyze,start=1):
    soft_res = soft_cls.forward(text)
    bert_res = bert_cls.forward(text)
    print(f"Text {i}. soft label: {soft_res}, bert label: {bert_res}")

Text 1. soft label: tensor([6.4218e-05]), bert label: tensor([[0.5429]])
Text 2. soft label: tensor([0.0723]), bert label: tensor([[0.7223]])
Text 3. soft label: tensor([0.0757]), bert label: tensor([[0.5027]])
Text 4. soft label: tensor([0.4062]), bert label: tensor([[0.5116]])
